<h1>Shark Attacks Data Analysis</h1>


Lets import all the libraries in use and the .py file with all the functions I am going to use to clean the Data.

In [1]:
#Libraries
import pandas as pd
import numpy as np
import math,datetime

#My Functions
import src.limpieza_texto as lt
import src.moonphase as moon


Open csv file and create a copy

In [2]:
data = pd.read_csv("data/attacks.csv", encoding = "ISO-8859-1")

#This shows me all the columns in the dataset when I do df.head():
pd.set_option('display.max_columns', None)
df = data.copy()
df.shape


(25723, 24)

Here, I'll take a look at the data and come up with some interesting hypotheses to try and prove with charts and data.  Data cleaning will depend on what information I will need and not need. 

In [3]:
df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,Age,Injury,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,F,57,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,"R. Collier, GSAF",2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,Adyson McNeely,F,11,Minor injury to left thigh,N,14h00 -15h00,NaN,"K.McMurray, TrackingSharks.com",2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,M,48,Injury to left lower leg from surfboard skeg,N,07h45,NaN,"K.McMurray, TrackingSharks.com",2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,male,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,"B. Myatt, GSAF",2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,Gustavo Ramos,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",A .Kipper,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2018.06.04,2018.06.04,6299.0,NaN,NaN


Hypotheses:.<br>
Make a map showing the countries with the most shark attacks.<br>
Did age play a factor in shark attack deaths? How about sex?<br>
Show the shark attacks counts over time. <br>
What activities resulted in the most shark?<br>
Where shark attacks more likely if they were provoked?<br>
Did shark attacks increase after the film Jaws came out?


Get rid of all the columns I wont need for my study. Some columns need renaming:

In [4]:
df_cleaned = df.drop(columns=["Case Number","Unnamed: 22", "Unnamed: 23", "Name", "pdf", "href formula","href","original order","Case Number.1", "Case Number.2", "Investigator or Source"])
names = list(df_cleaned.columns)
names

['Date',
 'Year',
 'Type',
 'Country',
 'Area',
 'Location',
 'Activity',
 'Sex ',
 'Age',
 'Injury',
 'Fatal (Y/N)',
 'Time',
 'Species ']

In [5]:
df_cleaned.rename(columns={'Species ': 'Species', 'Sex ': 'Sex', 'Fatal (Y/N)': 'Fatal' }, inplace=True)


If a row in the data contains all NaN values then delete those rows.  Also, I'll remove any rows that are duplicates and contain the sae iformation.  This will delete a lot of rows:

In [6]:
df_cleaned.dropna(axis = 0, how = 'all', inplace = True)
df_cleaned.drop_duplicates(subset=list(df_cleaned.columns), inplace = True)
df_cleaned.shape

(6295, 13)

A lot of data needs to be corrected and unified.  Next I'll apply some functions I implemented in my limpieza_textos.py file to clean this data:

In [11]:
df_cleaned['Age'] = df_cleaned['Age'].apply(lt.clean_age)
df_cleaned['Fatal'] = df_cleaned['Fatal'].apply(lt.clean_fatal)
df_cleaned['Sex'] = df_cleaned['Sex'].apply(lt.clean_sex)

# I dont really need to show years before the 1900's, so I'll create a new colum called Modern Year 
# and fill it in with the value from the year column:
df_cleaned['Year Modern'] = df_cleaned[df_cleaned['Year']>1900]['Year']

#Moon Phases
df_cleaned["Date Cleaned"] = df_cleaned['Date'].apply(lt.clean_date)
df_cleaned["Moon"] = df_cleaned['Date Cleaned'].apply(lt.get_moon)


In [12]:
df_cleaned.reset_index(drop=True)
df_cleaned.head()

,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,Injury,Fatal,Time,Species,Year Modern,Date Cleaned,Moon
0,25-Jun-2018,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,F,57.0,"No injury to occupant, outrigger canoe and pad...",N,18h00,White shark,2018.0,2018-06-25,Waxing Gibbous
1,18-Jun-2018,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,F,11.0,Minor injury to left thigh,N,14h00 -15h00,NaN,2018.0,2018-06-18,Waxing Crescent
2,09-Jun-2018,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,M,48.0,Injury to left lower leg from surfboard skeg,N,07h45,NaN,2018.0,2018-06-09,Waning Crescent
3,08-Jun-2018,2018.0,Unprovoked,AUSTRALIA,New South Wales,Arrawarra Headland,Surfing,M,NaN,Minor injury to lower leg,N,NaN,2 m shark,2018.0,2018-06-08,Last Quarter
4,04-Jun-2018,2018.0,Provoked,MEXICO,Colima,La Ticla,Free diving,M,NaN,Lacerations to leg & hand shark PROVOKED INCIDENT,N,NaN,"Tiger shark, 3m",2018.0,2018-06-04,Waning Gibbous


Lets export this data as a new .csv file:

In [13]:
df_cleaned.to_csv("data/attacks_clean.csv")